# Bayesian inference on F1 data : Build bigquery database from .csv files

In [44]:
import pandas as pd

## 1. Download data from Kaggle

Credit goes to Kaggle user [Chris G](https://www.kaggle.com/cjgdev) for providing the [F1 dataset](https://www.kaggle.com/datasets/cjgdev/formula-1-race-data-19502017) on Kaggle. This data is available under the [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

Go to (or create) your Kaggle account, download an API key .json file, and place it in ~/.kaggle/. Then download the F1 data:

In [45]:
#  ## install kaggle
#  #!pip install kaggle
#  
#  # download data, unzip and place csv files into directory
#  !kaggle datasets download -d cjgdev/formula-1-race-data-19502017
#  !mkdir csv
#  !unzip -d csv formula-1-race-data-19502017.zip
#  !rm formula-1-race-data-19502017.zip

  0%|                                               | 0.00/4.58M [00:00<?, ?B/s]
100%|███████████████████████████████████████| 4.58M/4.58M [00:00<00:00, 168MB/s]
mkdir: cannot create directory ‘csv’: File exists
Archive:  formula-1-race-data-19502017.zip
replace csv/circuits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## 2. Inspect dataframes

In [46]:
# two of the .csv files are not UTF-8 encoded, so need to be opened, encoded correctly, and saved
circuits = pd.read_csv('csv/circuits.csv', encoding="ISO-8859-1").to_csv('csv/circuits.csv', index=False)
drivers = pd.read_csv('csv/drivers.csv', encoding="ISO-8859-1").to_csv('csv/drivers.csv', index=False)

In [47]:
circuits = pd.read_csv('csv/circuits.csv')
circuits

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÃ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
68,69,americas,Circuit of the Americas,Austin,USA,30.13280,-97.64110,NaN,http://en.wikipedia.org/wiki/Circuit_of_the_Am...
69,70,red_bull_ring,Red Bull Ring,Spielburg,Austria,47.21970,14.76470,NaN,http://en.wikipedia.org/wiki/Red_Bull_Ring
70,71,sochi,Sochi Autodrom,Sochi,Russia,43.40570,39.95780,NaN,http://en.wikipedia.org/wiki/Sochi_Autodrom
71,72,port_imperial,Port Imperial Street Circuit,New Jersey,USA,40.77690,-74.01110,NaN,http://en.wikipedia.org/wiki/Port_Imperial_Str...


In [48]:
constructor_results = pd.read_csv('csv/constructorResults.csv')
constructor_results

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,NaN
1,2,18,2,8.0,NaN
2,3,18,3,9.0,NaN
3,4,18,4,5.0,NaN
4,5,18,5,2.0,NaN
...,...,...,...,...,...
11137,15635,988,1,2.0,NaN
11138,15636,988,3,1.0,NaN
11139,15637,988,210,0.0,NaN
11140,15638,988,15,0.0,NaN


In [49]:
constructors = pd.read_csv('csv/constructors.csv', )
constructors

,constructorId,constructorRef,name,nationality,url,Unnamed: 5
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,NaN
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,NaN
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_F1,NaN
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,NaN
...,...,...,...,...,...,...
203,206,marussia,Marussia,Russian,http://en.wikipedia.org/wiki/Marussia_F1,NaN
204,207,caterham,Caterham,Malaysian,http://en.wikipedia.org/wiki/Caterham_F1,NaN
205,208,lotus_f1,Lotus F1,British,http://en.wikipedia.org/wiki/Lotus_F1,NaN
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport,NaN


In [50]:
constructor_standings = pd.read_csv('csv/constructorStandings.csv')
constructor_standings

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins,Unnamed: 7
0,1,18,1,14.0,1,1,1,NaN
1,2,18,2,8.0,3,3,0,NaN
2,3,18,3,9.0,2,2,0,NaN
3,4,18,4,5.0,4,4,0,NaN
4,5,18,5,2.0,5,5,0,NaN
...,...,...,...,...,...,...,...,...
11891,26928,988,5,53.0,7,7,0,NaN
11892,26929,988,4,57.0,6,6,0,NaN
11893,26930,988,15,5.0,10,10,0,NaN
11894,26931,988,1,30.0,9,9,0,NaN


In [51]:
drivers = pd.read_csv('csv/drivers.csv')
drivers

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
837,838,vandoorne,2.0,VAN,Stoffel,Vandoorne,26/03/1992,Belgian,http://en.wikipedia.org/wiki/Stoffel_Vandoorne
838,839,ocon,31.0,OCO,Esteban,Ocon,17/09/1996,French,http://en.wikipedia.org/wiki/Esteban_Ocon
839,840,stroll,18.0,STR,Lance,Stroll,29/10/1998,Canadian,http://en.wikipedia.org/wiki/Lance_Stroll
840,841,giovinazzi,36.0,GIO,Antonio,Giovinazzi,14/12/1993,Italian,http://en.wikipedia.org/wiki/Antonio_Giovinazzi


In [52]:
driver_standings = pd.read_csv('csv/driverStandings.csv')
driver_standings

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0
...,...,...,...,...,...,...,...
31721,68604,988,836,5.0,18,18,0
31722,68605,988,18,0.0,24,24,0
31723,68606,988,814,0.0,25,25,0
31724,68607,988,842,0.0,21,21,0


In [53]:
lap_times = pd.read_csv('csv/lapTimes.csv')
lap_times

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342
...,...,...,...,...,...,...
426628,988,825,50,13,1:43.928,103928
426629,988,825,51,13,1:44.138,104138
426630,988,825,52,13,1:43.934,103934
426631,988,825,53,13,1:44.164,104164


In [54]:
pit_stops = pd.read_csv('csv/pitStops.csv')
pit_stops

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842
...,...,...,...,...,...,...,...
6246,988,828,1,30,17:56:57,22.176,22176
6247,988,839,1,31,17:57:47,21.865,21865
6248,988,832,1,31,17:57:49,21.956,21956
6249,988,154,1,32,17:59:52,21.850,21850


In [55]:
qualifying = pd.read_csv('csv/qualifying.csv')
qualifying

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
7511,7535,988,154,210,8,16,1:39.516,NaN,NaN
7512,7536,988,842,5,10,17,1:39.724,NaN,NaN
7513,7537,988,836,15,94,18,1:39.930,NaN,NaN
7514,7538,988,828,15,9,19,1:39.994,NaN,NaN


In [56]:
races = pd.read_csv('csv/races.csv')
races

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...,...
992,1005,2018,17,22,Japanese Grand Prix,2018-10-07,05:00:00,http://en.wikipedia.org/wiki/2018_Japanese_Gra...
993,1006,2018,18,69,United States Grand Prix,2018-10-21,19:00:00,http://en.wikipedia.org/wiki/2018_United_State...
994,1007,2018,19,32,Mexican Grand Prix,2018-10-28,19:00:00,http://en.wikipedia.org/wiki/2018_Mexican_Gran...
995,1008,2018,20,18,Brazilian Grand Prix,2018-11-11,16:00:00,http://en.wikipedia.org/wiki/2018_Brazilian_Gr...


In [57]:
results = pd.read_csv('csv/results.csv')
results

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23772,23777,988,842,5,10.0,17,16.0,16,16,0.0,54,NaN,NaN,33.0,16.0,01:43.8,192.542,11
23773,23778,988,828,15,9.0,19,17.0,17,17,0.0,54,NaN,NaN,36.0,15.0,01:43.6,193.057,11
23774,23779,988,840,3,18.0,15,18.0,18,18,0.0,54,NaN,NaN,52.0,6.0,01:42.3,195.402,11
23775,23780,988,832,4,55.0,12,NaN,R,19,0.0,31,NaN,NaN,26.0,14.0,01:43.4,193.41,36


In [58]:
seasons = pd.read_csv('csv/seasons.csv')
seasons

,year,url
0,2009,http://en.wikipedia.org/wiki/2009_Formula_One_...
1,2008,http://en.wikipedia.org/wiki/2008_Formula_One_...
2,2007,http://en.wikipedia.org/wiki/2007_Formula_One_...
3,2006,http://en.wikipedia.org/wiki/2006_Formula_One_...
4,2005,http://en.wikipedia.org/wiki/2005_Formula_One_...
...,...,...
64,2014,http://en.wikipedia.org/wiki/2014_Formula_One_...
65,2015,http://en.wikipedia.org/wiki/2015_Formula_One_...
66,2016,https://en.wikipedia.org/wiki/2016_Formula_One...
67,2017,https://en.wikipedia.org/wiki/2017_Formula_One...


In [59]:
status = pd.read_csv('csv/status.csv')
status

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine
...,...,...
129,132,ERS
130,133,+49 Laps
131,134,+38 Laps
132,135,Brake duct


## 3. Tidy up dataframes

### a) Faulty cell

In [60]:
results.at[23764, 'fastestLapSpeed'] = pd.NA

### b) Drop junk/redundant columns

- junk

In [61]:
constructors['Unnamed: 5'].value_counts(dropna=False)

NaN    208
Name: Unnamed: 5, dtype: int64

In [62]:
constructors = constructors.drop('Unnamed: 5', axis=1)

In [63]:
constructor_standings['Unnamed: 7'].value_counts(dropna=False)

NaN    11896
Name: Unnamed: 7, dtype: int64

In [64]:
constructor_standings = constructor_standings.drop('Unnamed: 7', axis=1)

- redundant

In [65]:
assert constructor_standings['position'].all() == constructor_standings['positionText'].all()
constructor_standings = constructor_standings.drop('positionText', axis=1)

In [66]:
assert driver_standings['position'].all() == driver_standings['positionText'].all()
driver_standings = driver_standings.drop('positionText', axis=1)

In [67]:
results = results.drop('position', axis=1)
pit_stops = pit_stops.drop('duration', axis=1)
lap_times = lap_times.drop('time', axis=1)
results = results.drop('time', axis=1)

### c) Remove special characters and trailing whitespaces

In [68]:
import regex

# set of special characters to be removed from columns
chars = regex.compile(r'[^a-zA-Z !@#$%&*_+-=|\:";<>,./()[\]{}\']')

In [69]:
dataframes = [circuits,
              constructor_results,
              constructors,
              constructor_standings,
              drivers,
              driver_standings,
              lap_times,
              pit_stops,
              qualifying,
              races,
              results,
              seasons,
              status
             ]

# remove special characters
for dataframe in dataframes:
    for column in dataframe.columns:
        if dataframe[column].dtype == str:
            dataframe[column] = dataframe[column].apply(lambda x: chars.sub('', x))

### d) dtype changes

In [70]:
drivers['number'] = drivers['number'].astype(pd.Int64Dtype())
results['number'] = results['number'].astype(pd.Int64Dtype())
results['milliseconds'] = results['milliseconds'].astype(pd.Int64Dtype())
results['fastestLap'] = results['fastestLap'].astype(pd.Int64Dtype())
results['rank'] = results['rank'].astype(pd.Int64Dtype())

### e) Rename columns to make JOIN statements predictable and consistent

In [71]:
circuits.rename(columns={'circuitId': 'id',
                         'circuitRef': 'circuit_ref',
                        }, inplace=True)

In [72]:
constructor_results.rename(columns={'constructorResultsId': 'id',
                                    'raceId': 'race_id',
                                    'constructorId': 'constructor_id',
                                   }, inplace=True)

In [73]:
constructors.rename(columns={'constructorId': 'id',
                             'constructorRef': 'constructor_ref',
                             }, inplace=True)

In [74]:
constructor_standings.rename(columns={'constructorStandingsId': 'id',
                                      'raceId': 'race_id',
                                      'constructorId': 'constructor_id',
                                      }, inplace=True
                            )

In [75]:
drivers.rename(columns={'driverId': 'id',
                        'driverRef': 'driver_ref',
                       }, inplace=True
              )

In [76]:
driver_standings.rename(columns={'driverStandingsId': 'id',
                                 'raceId': 'race_id',
                                 'driverId': 'driver_id',                                 
                                }, inplace=True
                       )

In [77]:
lap_times.rename(columns={'raceId': 'race_id',
                          'driverId': 'driver_id',
                         }, inplace=True
                )

In [78]:
pit_stops.rename(columns={'raceId': 'race_id',
                          'driverId': 'driver_id',
                         }, inplace=True
                )

In [79]:
qualifying.rename(columns={'qualifyId': 'id',
                           'raceId': 'race_id',
                           'driverId': 'driver_id',
                           'constructorId': 'constructor_id',
                         }, inplace=True
                )

In [80]:
races.rename(columns={'raceId': 'id',
                      'circuitId': 'circuit_id',
                     }, inplace=True
                )

In [81]:
results.rename(columns={'resultId': 'id',
                        'raceId': 'race_id',
                        'driverId': 'driver_id',
                        'constructorId': 'constructor_id',
                        'positionText': 'position',
                        'positionOrder': 'position_order',
                        'fastestLap': 'fastest_lap',
                        'fastestLapTime': 'fastest_lap_time',
                        'fastestLapSpeed': 'fastest_lap_speed',
                        'statusId': 'status_id',
                     }, inplace=True
                )

In [82]:
status.rename(columns={'statusId': 'id',
                     }, inplace=True
                )

## 4. Save updated dataframes

In [83]:
circuits.to_csv('csv/circuits_cleaned.csv', index=False)
constructor_results.to_csv('csv/constructor_results_cleaned.csv', index=False)
constructors.to_csv('csv/constructors_cleaned.csv', index=False)
constructor_standings.to_csv('csv/constructor_standings_cleaned.csv', index=False)
drivers.to_csv('csv/drivers_cleaned.csv', index=False)
driver_standings.to_csv('csv/driver_standings_cleaned.csv', index=False)
lap_times.to_csv('csv/lap_times_cleaned.csv', index=False)
pit_stops.to_csv('csv/pit_stops_cleaned.csv', index=False)
qualifying.to_csv('csv/qualifying_cleaned.csv', index=False)
races.to_csv('csv/races_cleaned.csv', index=False)
results.to_csv('csv/results_cleaned.csv', index=False)
seasons.to_csv('csv/seasons_cleaned.csv', index=False)
status.to_csv('csv/status_cleaned.csv', index=False)

## 5. Create bigquery tables

In [85]:
# joinable column groups in format <db_name>.<table_name>.<column_name>
id_columns = (('f1.circuits.id',
               'f1.races.circuit_id'),
              
              ('f1.constructor_results.id'), 
              
              ('f1.constructor_results.race_id',
               'f1.constructor_standings.race_id',
               'f1.driver_standings.race_id',
               'f1.lap_times.race_id',
               'f1.pit_stops.race_id',
               'f1.qualifying.race_id',
               'f1.results.race_id',
               'f1.races.id'),
              
              ('f1.constructor_results.constructor_id',
               'f1.constructors.constructor_id',
               'f1.constructor_standings.constructor_id',
               'f1.qualifying.constructor_id',
               'f1.results.constructor_id',
               'f1.constructors.id'),

              ('f1.constructor_standings.id'),
              
              ('f1.drivers.id',
               'f1.driver_standings.driver_id',
               'f1.lap_times.driver_id',
               'f1.pit_stops.driver_id ',
               'f1.qualifying.driver_id',
               'f1.results.driver_id'),
              
              ('f1.driver_standings.driver_standings_id'),
              
              ('f1.qualifying.id'),

              ('f1.results.id'),
              
              ('f1.status.id',
               'f1.results.status_id'),
             )

# store this data to open it in another notebook
%store id_columns

Stored 'id_columns' (tuple)


In [42]:
# make bigquery database called "f1"
!bq mk f1

Dataset 'sdlfuhq30945yqw34lhi23u450:f1' successfully created.


In [43]:
# create bigquery tables
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.circuits csv/circuits_cleaned.csv \
circuitId:INTEGER,circuitRef:STRING,name:STRING,location:STRING,country:STRING,lat:FLOAT,lng:FLOAT,alt:FLOAT,url:STRING

!bq load --source_format=CSV --skip_leading_rows 1 \
f1.constructor_results csv/constructor_results_cleaned.csv \
id:INTEGER,race_id:INTEGER,constructor_id:INTEGER,points:FLOAT,status:STRING         
                    
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.constructors csv/constructors_cleaned.csv \
id:INTEGER,constructor_ref:STRING,name:STRING,nationality:STRING,url:STRING
                    
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.constructor_standings csv/constructor_standings_cleaned.csv \
id:INTEGER,race_id:INTEGER,constructor_id:INTEGER,points:FLOAT,position:INTEGER,wins:INTEGER
                        
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.drivers csv/drivers_cleaned.csv \
id:INTEGER,driver_ref:STRING,number:INTEGER,code:STRING,forename:STRING,surname:STRING,dob:STRING,nationality:STRING,url:STRING
                                    
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.driver_standings csv/driver_standings_cleaned.csv \
id:INTEGER,race_id:INTEGER,driver_id:INTEGER,points:FLOAT,position:INTEGER,wins:INTEGER

!bq load --source_format=CSV --skip_leading_rows 1 \
f1.lap_times csv/lap_times_cleaned.csv \
race_id:INTEGER,driver_id:INTEGER,lap:INTEGER,position:INTEGER,milliseconds:INTEGER

!bq load --source_format=CSV --skip_leading_rows 1 \
f1.pit_stops csv/pit_stops_cleaned.csv \
race_id:INTEGER,driver_id:INTEGER,stop:INTEGER,lap:INTEGER,time:STRING,milliseconds:INTEGER
                        
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.qualifying csv/qualifying_cleaned.csv \
id:INTEGER,race_id:INTEGER,driver_id:INTEGER,constructor_id:INTEGER,number:INTEGER,position:INTEGER,q1:STRING,q2:STRING,q3:STRING

!bq load --source_format=CSV --skip_leading_rows 1 \
f1.races csv/races_cleaned.csv \
id:INTEGER,year:INTEGER,round:INTEGER,circuit_id:INTEGER,name:STRING,date:STRING,time:STRING,url:STRING
                                
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.results csv/results_cleaned.csv \
id:INTEGER,race_id:INTEGER,driver_id:INTEGER,constructor_id:INTEGER,number:INTEGER,grid:INTEGER,position:STRING,position_order:STRING,points:FLOAT,laps:INTEGER,milliseconds,fastest_lap:INTEGER,rank:INTEGER,fastest_lap_time:STRING,fastest_lap_speed:FLOAT,status_id:INTEGER
                                                            
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.seasons csv/seasons_cleaned.csv \
year:INTEGER,url:STRING
        
!bq load --source_format=CSV --skip_leading_rows 1 \
f1.status csv/status_cleaned.csv \
id:INTEGER,status:STRING

Upload complete.
Waiting on bqjob_r72d66370eec360db_00000181a023916b_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r401dde3e97c47841_00000181a023a6de_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r4d705fd5a4b5c37a_00000181a023bc93_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r6d903ce73aecaafb_00000181a023d265_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r76edc039f0c14406_00000181a023e75d_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r72b4890a55890ae1_00000181a023fc51_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r7744dc53c9d71271_00000181a0241196_1 ... (2s) Current status: DONE   
Upload complete.
Waiting on bqjob_r119e76958c2c2fff_00000181a0242b6a_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r222c3361bc98a4a_00000181a024417c_1 ... (1s) Current status: DONE   
Upload complete.
Waiting on bqjob_r19423583d08f45f8_00000181a0245